In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 10
img_rows, img_cols = 28, 28

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= X.max()
#x_test /= X.max()

#Save y_train/test before onehot encoding
np.save('T22Audio/train_features_Y.npy',y_train)
np.save('T22Audio/test_features_Y.npy',y_test)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
def Conv():
    input_img = Input(shape=input_shape)  # adapt this if using `channels_first` image data format   
    e1 = Conv2D(16, (3, 3),strides=(2, 2), activation='relu', padding='valid')(input_img)
    e2 = MaxPooling2D(pool_size=(2, 2), padding='same')(e1)
    e3 = Conv2D(32, (3, 3), activation='relu', padding='same')(e2)
    e4 = Conv2D(64, (3, 3), activation='relu', padding='same')(e3)
    e5 = Conv2D(128, (3, 3), activation='relu', padding='same')(e4)
    #e6 = MaxPooling2D(pool_size=(2, 2), padding='same')(e5)
    e7 = Flatten()(e5)
    e8 = Dense(300, activation='relu')(e7)
    e9 = Dense(450, activation='relu')(e8)
    e10 = Dense(600, activation='relu')(e9)
    return Model(input_img, e10)


def FCN():
    input_net = Input(shape=(600,))  # adapt this if using `channels_first` image data format   
    d1 = Dense(num_classes, activation='softmax')(input_net)
    return Model(input_net, d1)

x = Input(shape=input_shape)
modelFCN=FCN()
modelConv=Conv()
# make the model:
model = Model(x, modelFCN(modelConv(x)))

In [7]:
model.compile(keras.optimizers.Adam(lr=0.001), 
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.2358 - acc: 0.9352 - val_loss: 0.0691 - val_acc: 0.9776
Epoch 2/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0592 - acc: 0.9820 - val_loss: 0.0580 - val_acc: 0.9827
Epoch 3/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0419 - acc: 0.9865 - val_loss: 0.0509 - val_acc: 0.9848
Epoch 4/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0368 - acc: 0.9889 - val_loss: 0.0599 - val_acc: 0.9842
Epoch 5/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0313 - acc: 0.9905 - val_loss: 0.0648 - val_acc: 0.9812
Epoch 6/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0277 - acc: 0.9916 - val_loss: 0.0505 - val_acc: 0.9858
Epoch 7/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0234 - acc: 0.9930 - val_loss: 0.0520 - val_acc

In [9]:
np.save('T22Audio/test_features_X.npy',modelConv.predict(x_test))
np.save('T22Audio/train_features_X.npy',modelConv.predict(x_train))